#WELCOME
In this Assignment, you'll use **Classifier** to predict the output. You are allowed to write/edit between the
```# START CODE HERE``` & ```# END CODE HERE``` only.<br>
Don't edit the test cells otherwise you'd be failed in the assignment.

Upload the ```utils.zip``` file before running the 1st cell.
[Demo](https://drive.google.com/file/d/1ebkQR6ikjGottmxa5KL83ZWdjTLvRxbm/view?usp=share_link)

**BEST OF LUCK**

# Dataset Loading and Pre-Processing

In [1]:
import pandas as pd
data = pd.read_csv("/Users/rahuljarpula/Desktop/ML Course/Machine-Learning-LS-24-main/Week 2/Assignment/Classifiers Assignment/content/weather_classification_data.csv")

## Data Info
```data.info``` gives info about data-types and number of null objects.  
There are no null data, no need to remove any.

In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13200 entries, 0 to 13199
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Temperature           13200 non-null  float64
 1   Humidity              13200 non-null  int64  
 2   Wind Speed            13200 non-null  float64
 3   Precipitation (%)     13200 non-null  float64
 4   Cloud Cover           13200 non-null  object 
 5   Atmospheric Pressure  13200 non-null  float64
 6   UV Index              13200 non-null  int64  
 7   Season                13200 non-null  object 
 8   Visibility (km)       13200 non-null  float64
 9   Location              13200 non-null  object 
 10  Weather Type          13200 non-null  object 
dtypes: float64(5), int64(2), object(4)
memory usage: 1.1+ MB


## Data Shuffling
Data Shuffling removes any bias present and creates uniform distribution.

In [3]:
from sklearn.utils import shuffle
data = shuffle(data)
data.head()

,Temperature,Humidity,Wind Speed,Precipitation (%),Cloud Cover,Atmospheric Pressure,UV Index,Season,Visibility (km),Location,Weather Type
5280,26.0,91,11.0,80.0,partly cloudy,998.77,2,Winter,2.5,inland,Rainy
7048,-6.0,82,10.0,90.0,overcast,985.04,0,Winter,1.5,inland,Snowy
7159,21.0,50,2.5,42.0,partly cloudy,1014.07,4,Winter,5.5,mountain,Cloudy
4952,20.0,69,13.5,34.0,partly cloudy,1014.07,4,Summer,5.5,inland,Cloudy
156,31.0,86,13.5,78.0,overcast,999.25,3,Winter,4.0,inland,Rainy


## Scaling and Labeling
Machine can't understand string so all the object datatypes need to be LABELED with integers.<br><br>
Why SCALING?<br>
You might have noticed one column contains data with large values (of order 1000) whereas some column contain small value data (of order 10). As a result, large value columns get more power/importance than others. To remove this disparity we scaled them to same order/values.

**CODE**

We need to find the columns with object datatypes.

HINT: Use ```data.select_dtypes().columns.tolist()```<br>Reference: [w3schools](https://www.w3schools.com/python/pandas/ref_df_select_dtypes.asp#:~:text=The%20select_dtypes()%20method%20returns,the%20specified%20dtype(s).&text=Note%3A%20You%20must%20specify%20at,you%20will%20get%20an%20error.)

In [4]:
# START CODE HERE
object_columns = data.select_dtypes(include='object').columns.tolist()
non_object_columns = data.select_dtypes(exclude='object').columns.tolist()
# END CODE HERE
print(f"Object Columns: {object_columns}\nNon Object Columns: {non_object_columns}")

Object Columns: ['Cloud Cover', 'Season', 'Location', 'Weather Type']
Non Object Columns: ['Temperature', 'Humidity', 'Wind Speed', 'Precipitation (%)', 'Atmospheric Pressure', 'UV Index', 'Visibility (km)']


**TEST** (DON'T EDIT THIS CELL)

In [5]:
# DON'T EDIT THIS CELL
if (object_columns == ['Cloud Cover', 'Season', 'Location', 'Weather Type'] and non_object_columns == ['Temperature', 'Humidity', 'Wind Speed', 'Precipitation (%)', 'Atmospheric Pressure', 'UV Index', 'Visibility (km)']):
  print(f"\033[32mTest Passed\033[0m")
else:
  print(f"\033[31mTest Failed\033[0m")

Test Passed


**CODE**

We need to convert the object-type columns data into integral labels and scale the non-object-type columns data.

HINT: Use ```ColumnTransformer()```<br>Reference: [scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html), [ChatGPT](https://chatgpt.com/c/7f3c1b02-5da0-4c34-bdb2-d999dccb9c2f)

In [6]:
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

# START CODE HERE

column_transformer =  ColumnTransformer(
    transformers=[
        ('object', OrdinalEncoder(), object_columns),
        ('non_object', StandardScaler(), non_object_columns)
    ]
)

# Fit and transform the data
data_scaled_labeled = column_transformer.fit_transform(data)

# END CODE HERE
all_columns = object_columns + non_object_columns

# Create the new DataFrame
data_scaled_labeled = pd.DataFrame(data_scaled_labeled, columns=all_columns)

**TEST** (DON'T EDIT THIS CELL)

In [7]:
# DON'T EDIT THIS CELL

data_scaled_labeled_check = pd.read_csv("/Users/rahuljarpula/Desktop/ML Course/Machine-Learning-LS-24-main/Week 2/Assignment/Classifiers Assignment/content/data_scaled_labeled_check.csv")
import numpy as np
data_scaled_labeled_values = np.sort(data_scaled_labeled.values,axis=0)
data_scaled_labeled_check_values = np.sort(data_scaled_labeled_check.values,axis=0)
if np.allclose(data_scaled_labeled_values, data_scaled_labeled_check_values, equal_nan=True):
  print(f"\033[32mTest Passed\033[0m")
else:
  print(f"\033[31mTest Failed\033[0m")

Test Passed


## Data Splitting

**CODE**

We need to drop the Weather Type column from data_scaled_labeled to get the X and pick Weather Type column to get Y

Use 90-10 split of data, i.e. 90% for training, 10% for testing

In [8]:
X = data_scaled_labeled.drop(columns=['Weather Type'])
y = data_scaled_labeled['Weather Type']

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, train_size = 0.9, random_state = 10)

**TEST** (DON'T EDIT THIS CELL)

In [9]:
# DON'T EDIT THIS CELL

if (len(X_train)==11880 and len(X_test)==1320):
  print(f"\033[32mTest Passed\033[0m")
else:
  print(f"\033[31mTest Failed\033[0m")

if (len(y_train)==11880 and len(y_test)==1320):
  print(f"\033[32mTest Passed\033[0m")
else:
  print(f"\033[31mTest Failed\033[0m")

Test Passed
Test Passed


In [10]:
X_train.head()

,Cloud Cover,Season,Location,Temperature,Humidity,Wind Speed,Precipitation (%),Atmospheric Pressure,UV Index,Visibility (km)
7017,2.0,3.0,1.0,-1.100192,0.410487,-1.133714,0.887629,-0.570133,-1.038715,-0.878846
12278,0.0,1.0,1.0,2.926113,0.608571,-0.916588,0.543291,0.426690,-0.779410,1.494076
2859,3.0,1.0,1.0,0.855442,0.311445,0.748042,-1.115794,-0.047258,-0.001493,0.010999
5495,3.0,3.0,0.0,0.222737,1.054259,1.037543,1.263271,0.312975,-1.038715,-1.323769
8469,3.0,1.0,0.0,0.682886,-0.332327,-0.048086,0.637201,0.168076,-0.779410,-0.285616


# Training and Testing
Use any suitable classifier to fit the training data and check the accuracy on test data

Don't forget to import that library

**CODE**

ACCURACY should be more than 90%,

1. Fit the model to X_train and y_train
2. Find the model score on X_test and y_test

In [11]:
# START CODE HERE

from sklearn.ensemble import RandomForestClassifier


model =  RandomForestClassifier(random_state=10)
model.fit(X_train,y_train)
score = model.score(X_test,y_test)

# END CODE HERE

print("Accuracy:", score*100)

Accuracy: 91.36363636363637


**TEST** (DON'T EDIT THIS CELL)

In [12]:
# DON'T EDIT THIS CELL

if (score>0.90):
  print(f"\033[32mTest Passed\033[0m")
else:
  print(f"\033[31mTest Failed\033[0m")

Test Passed


# Predicting Output (Here, Weather Type) (Optional)

In [13]:
input = X_test.head()
input

,Cloud Cover,Season,Location,Temperature,Humidity,Wind Speed,Precipitation (%),Atmospheric Pressure,UV Index,Visibility (km)
11923,0.0,2.0,2.0,0.280255,-2.016038,-0.265211,-1.209704,0.343353,1.035729,1.197460
2331,3.0,2.0,2.0,-0.352450,1.846594,-1.206089,1.701520,0.291469,-0.520104,-0.433923
217,2.0,3.0,2.0,-1.387785,-0.282806,0.313790,0.042435,-0.495129,-1.038715,-1.027154
1760,2.0,3.0,1.0,-1.445304,0.608571,1.399419,0.386773,-0.286248,-1.038715,-1.175461
3963,0.0,1.0,1.0,0.165218,-1.669392,-1.133714,-1.178401,0.463251,0.776424,1.345768


In [14]:
y = model.predict(input)
y

array([3., 0., 2., 2., 3.])

Here, the outputs are in numbers. We can perform inverse column transfer for getiing the string values.


In [15]:
#  using ord-encoder for obj - numeric and vice - versa
weather_type_encoder = OrdinalEncoder()
weather_type_encoder.fit(data[['Weather Type']]) # fitting obj

# numeric - obj
predicted_strings = weather_type_encoder.inverse_transform(y.reshape(-1, 1))
predicted_strings

array([['Sunny'],
       ['Cloudy'],
       ['Snowy'],
       ['Snowy'],
       ['Sunny']], dtype=object)